In [1]:
import numpy as np
from openqaoa.problems import TSP, Knapsack, MinimumVertexCover
import json
import os
import networkx as nx

In [2]:
def generate_results_filename(problem_type, num_qubits):
    if problem_type == 'Knapsack':
        problem_type = f"{problem_type}_{N_ITEMS}_items"
    return f"batch_Ising_data_{str(problem_type).replace(' ', '_')}_{num_qubits}q_.json"

In [ ]:
desiredProblemType = 'Knapsack' #options: 'TSP','Knapsack', 'MinimumVertexCover'
#make sure you typed in that mf desiredProblemType correctly
batchSize = 100
RESULTS_FOLDER = "isingBatches"


all_isings_list = []
for instance in range(batchSize):        
    if desiredProblemType == 'TSP':
        n_cities = 4  # Number of cities for TSP
        connection_probability = 1  # Probability for edge creation in gnp_random_graph
        # Ensure the graph is connected
        G = nx.generators.fast_gnp_random_graph(n=n_cities, p=connection_probability) 

        min_edge_weight = 1
        max_edge_weight = 10 
        rng_weights = np.random.default_rng() # Use a different seed or manage seeds as needed
        for (u, v) in G.edges():
            weight = int(rng_weights.integers(low=min_edge_weight, high=max_edge_weight, endpoint=True)) # endpoint=True includes high value
            G.edges[u, v]['weight'] = weight
        # A is the penalty for not visiting a city or visiting it multiple times.
        # B is the penalty for not having a valid tour structure (e.g. sub-tours or incorrect number of cities at a position)
        B_val = 1
        A_val = 15
        tsp_prob = TSP(G=G, A=A_val, B=B_val) # Using your specified A and B penalties
        isingProb = tsp_prob.qubo

    elif desiredProblemType == 'Knapsack':
        N_ITEMS = 4
        target_qubits = 6
        # Loop until an instance with the correct number of qubits is found
        while True:
            knapsack_prob = Knapsack.random_instance(n_items=N_ITEMS)
            isingProb = knapsack_prob.qubo
            if isingProb.n == target_qubits and knapsack_prob.weight_capacity!= 8:  # Ensure the weight capacity is not 16 to avoid weird edge cases where slack variables cant describe weight capacity
                break

    elif desiredProblemType == 'MinimumVertexCover':
        G = nx.generators.fast_gnp_random_graph(n=9, p=0.6)
        mvc_prob = MinimumVertexCover(G, field=1, penalty=10)
        isingProb = mvc_prob.qubo
    
    print(f"Generated Ising model for {desiredProblemType} instance {instance + 1} of {batchSize}", end="\r")
    ising_dict = isingProb.asdict()
    current_ising_data = {
        "instance_id": instance + 1, # Add an identifier for each QUBO instance
        "terms": ising_dict["terms"],
        "weights": ising_dict["weights"],
        "constant": ising_dict.get("constant", 0.0),
        "problem_type": ising_dict.get("problem_instance", {}).get("problem_type", "unknown"),
        # Optional: include more problem-specific details if needed for later analysis, can add the number of qubits/problem size
    }

    if desiredProblemType == 'Knapsack':
        current_ising_data['weight_capacity'] = knapsack_prob.weight_capacity #necessary for custom inital state generation
 
    all_isings_list.append(current_ising_data) 

all_indices = []
terms = ising_dict["terms"]
for term_group in terms:
    for idx in term_group:
        all_indices.append(idx)
num_qubits = max(all_indices) + 1

file_name = generate_results_filename(desiredProblemType, num_qubits)
results_filename_with_path = os.path.join(RESULTS_FOLDER, file_name)

with open(results_filename_with_path, "w") as f:
    f.write("[\n") # Start of the JSON array
    for i, single_ising_data in enumerate(all_isings_list):
        ising_json_string = json.dumps(single_ising_data)
        f.write("  " + ising_json_string)
        
        if i < len(all_isings_list) - 1:
            f.write(",\n")
        else:
            f.write("\n")
            
    f.write("]\n") 

print(f"\nBatch of {len(all_isings_list)} Ising models saved to {results_filename_with_path}")
    

Generated Ising model for Knapsack instance 100 of 100
Batch of 100 Ising models saved to isingBatches/batch_Ising_data_Knapsack_4_items_7q_.json
